In [1]:
import pandas as pd

In [2]:
# Loading data
data = pd.read_csv('Prodigy University Dataset.csv')
# Split the data into features (X) and target (y)
data.head()

,sat_sum,hs_gpa,fy_gpa
0,508,3.40,3.18
1,488,4.00,3.33
2,464,3.75,3.25
3,380,3.75,2.42
4,428,4.00,2.63


In [3]:
# Converting data to numpy
X = data[['sat_sum', 'hs_gpa']].values
# reshape the fy_gpa into a 2D array with [data_size] rows and 1 column
y = data['fy_gpa'].values.reshape(-1, 1)
print(X.shape)
print(y.shape)

(1000, 2)
(1000, 1)


In [4]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.preprocessing import StandardScaler

# Normalize the features so that it is easier to train the data
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.fit_transform(X_test)

In [6]:
X_train.shape

(800, 2)

In [7]:
import torch
# Convert numpy to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [8]:
import torch.nn as nn

In [10]:
# Building model with 2 neurons
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(),
    nn.Linear(2, 1)
)

In [11]:
# Forward Propagation
preds = model(X_train_tensor)

In [12]:
from torch.nn import MSELoss

In [13]:
preds[:5]

tensor([[0.1223],
        [0.1129],
        [0.0293],
        [0.1440],
        [0.0043]], grad_fn=<SliceBackward0>)

In [14]:
# Calculating Loss
criterion = MSELoss()
loss = criterion(preds, y_train_tensor)
print(loss)
# to learners: You may get different values

tensor(5.9385, grad_fn=<MseLossBackward0>)


# Comparing predictions on X_train with Target

In [15]:
preds[:5]

tensor([[0.1223],
        [0.1129],
        [0.0293],
        [0.1440],
        [0.0043]], grad_fn=<SliceBackward0>)

In [16]:
y_train_tensor[:5]

tensor([[2.0000],
        [3.1100],
        [1.6300],
        [3.0200],
        [1.5500]])

In [17]:
model[0].weight

Parameter containing:
tensor([[-0.2010, -0.5799],
        [ 0.1211,  0.3072]], requires_grad=True)

In [18]:
model[2].weight

Parameter containing:
tensor([[-0.3010,  0.5979]], requires_grad=True)

---

Run all the cells above till here!

# Optimization and Backpropogation

In [19]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [20]:
loss.backward()

In [21]:
optimizer.step()

In [22]:
model[0].weight

Parameter containing:
tensor([[-0.2010, -0.5799],
        [ 0.1211,  0.3072]], requires_grad=True)

In [23]:
model[2].weight

Parameter containing:
tensor([[-0.2991,  0.6007]], requires_grad=True)

In [24]:
from torch.utils.data import TensorDataset, DataLoader

In [25]:
train_data = TensorDataset(X_train_tensor, y_train_tensor)

In [26]:
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(),
    nn.Linear(2, 1)
)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [27]:
# performance on train  and test sets  before training
train_loss = criterion(model(X_train_tensor), y_train_tensor).item()
test_loss = criterion(model(X_test_tensor), y_test_tensor).item()
print(f'Without Training:\nTrain Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Without Training:
Train Loss: 9.6453, Test Loss: 9.9598


In [28]:
# Looking at predictions
model(X_train_tensor)[:5]

tensor([[-0.7091],
        [-0.6832],
        [-0.4896],
        [-0.6274],
        [-0.5459]], grad_fn=<SliceBackward0>)

# Stochastic Gradient Descent

In [29]:
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
# Execute the training loop
for epoch in range(10):
    for X_batch, y_batch in train_loader:
        # Forward pass
        pred = model(X_batch)
        loss = criterion(pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = criterion(model(X_train_tensor), y_train_tensor).item()
    # print(epoch,': ', train_loss)
    test_loss = criterion(model(X_test_tensor), y_test_tensor).item()
    print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch 1: Train Loss: 0.7475, Test Loss: 0.8321
Epoch 2: Train Loss: 0.5839, Test Loss: 0.6454
Epoch 3: Train Loss: 0.5049, Test Loss: 0.5622
Epoch 4: Train Loss: 0.4530, Test Loss: 0.5094
Epoch 5: Train Loss: 0.4184, Test Loss: 0.4717
Epoch 6: Train Loss: 0.3955, Test Loss: 0.4496
Epoch 7: Train Loss: 0.3803, Test Loss: 0.4325
Epoch 8: Train Loss: 0.3698, Test Loss: 0.4225
Epoch 9: Train Loss: 0.3622, Test Loss: 0.4166
Epoch 10: Train Loss: 0.3575, Test Loss: 0.4113


In [30]:
# Looking at predictions
model(X_train_tensor)[:5]

tensor([[2.3234],
        [2.3198],
        [2.1826],
        [2.5166],
        [2.0564]], grad_fn=<SliceBackward0>)

# Batch Gradient Descent

In [31]:
# Reinitialising model weights
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(),
    nn.Linear(2, 1)
)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [32]:
train_loader = DataLoader(train_data, batch_size=800, shuffle=True) #800 is the number of samples in train set
# Execute the training loop
for epoch in range(1000): # increasing the epochs for effective training
    for X_batch, y_batch in train_loader:
        # Forward pass
        pred = model(X_batch)
        loss = criterion(pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0: # printing after every 100 epochs
        train_loss = criterion(model(X_train_tensor), y_train_tensor).item()
        # print(epoch,': ', train_loss)
        test_loss = criterion(model(X_test_tensor), y_test_tensor).item()
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch 100: Train Loss: 3.2054, Test Loss: 3.4044
Epoch 200: Train Loss: 2.0763, Test Loss: 2.2405
Epoch 300: Train Loss: 1.4520, Test Loss: 1.5896
Epoch 400: Train Loss: 1.1067, Test Loss: 1.2242
Epoch 500: Train Loss: 0.9136, Test Loss: 1.0159
Epoch 600: Train Loss: 0.8025, Test Loss: 0.8935
Epoch 700: Train Loss: 0.7353, Test Loss: 0.8179
Epoch 800: Train Loss: 0.6915, Test Loss: 0.7679
Epoch 900: Train Loss: 0.6605, Test Loss: 0.7322
Epoch 1000: Train Loss: 0.6365, Test Loss: 0.7047


# Mini-Batch Gradient Descent

In [33]:
# Reinitialising model weights
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(),
    nn.Linear(2, 1)
)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [34]:
train_loader = DataLoader(train_data, batch_size= 64, shuffle=True) #800 is the number of samples in train set
# Execute the training loop
for epoch in range(500): # increasing the epochs for effective training
    for X_batch, y_batch in train_loader:
        # Forward pass
        pred = model(X_batch)
        loss = criterion(pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 50 == 0: # printing after every 100 epochs
        train_loss = criterion(model(X_train_tensor), y_train_tensor).item()
        # print(epoch,': ', train_loss)
        test_loss = criterion(model(X_test_tensor), y_test_tensor).item()
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

Epoch 50: Train Loss: 0.5837, Test Loss: 0.6660
Epoch 100: Train Loss: 0.4364, Test Loss: 0.4916
Epoch 150: Train Loss: 0.4097, Test Loss: 0.4620
Epoch 200: Train Loss: 0.3913, Test Loss: 0.4431
Epoch 250: Train Loss: 0.3780, Test Loss: 0.4301
Epoch 300: Train Loss: 0.3685, Test Loss: 0.4210
Epoch 350: Train Loss: 0.3618, Test Loss: 0.4146
Epoch 400: Train Loss: 0.3569, Test Loss: 0.4102
Epoch 450: Train Loss: 0.3534, Test Loss: 0.4071
Epoch 500: Train Loss: 0.3510, Test Loss: 0.4052
